<a href="https://colab.research.google.com/github/Hassan-DS507/data-science-notebooks/blob/main/Task_1_for_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Tom Clinic Data Cleaning Project

##  Objective
Clean and prepare the provided clinic dataset to make it ready for analysis by:
- Handling missing values
- Fixing inconsistent text entries
- Removing duplicates
- Ensuring correct data types

##  Step 1: Import Necessary Libraries

In [1]:
# Data Handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


# Step 2: Load the Datasetet
We load the dataset and take an initial look at its structure.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/tasks_to_dataset/01JT7BFHK057AQS04QAAWHCWNX (1).csv')
df.head()

,Invoice_ID,Date,Customer_Name,Product,Brand,Quantity,Unit_Price,Branch,Payment_Method,Total_Price
0,PET-7000,01/26/2025,Salma,Pet Shampoo,Royal Canin,1,120,Alexandria,Visa,120
1,PET-7001,2025-02-20,NaN,Pet Carrier,Hill's,3,350,6th October,Mobile Wallet,1050
2,PET-7002,2024-06-22,Salma,Canned Pet Food,Royal Canin,1,70,Cairo,Visa,70
3,PET-7003,2025-04-12,Salma,Pet Carrier,Purina,3,400,Port Said,Cash,1200
4,PET-7004,2024-12-26,Reem,Scratching Post,Purina,2,250,6th October,NaN,500


In [4]:
df.sample(5)

,Invoice_ID,Date,Customer_Name,Product,Brand,Quantity,Unit_Price,Branch,Payment_Method,Total_Price
683,PET-7683,2025-03-30,Tarek,Pet Bed,Friskies,3,600,Alexandria,Cash,1800
784,PET-7784,2024-12-05,Tarek,Scratching Post,Trixie,1,300,Port Said,Cash,300
175,PET-7175,2024-09-27,Tarek,Leash,Purina,3,150,Cairo,Visa,450
726,PET-7726,2024-03-26,Tarek,Pet Carrier,Purina,3,300,Port Said,Visa,900
2155,PET-9155,2024-03-11,Reem,Cat Litter,Pedigree,1,70,6th October,Installments,70


# Step 3: Explore the Data

In [5]:
df.shape

(2600, 10)

In [6]:
df.columns

Index(['Invoice_ID', 'Date', 'Customer_Name', 'Product', 'Brand', 'Quantity',
       'Unit_Price', 'Branch', 'Payment_Method', 'Total_Price'],
      dtype='object')

In [7]:
df.nunique().sort_values(ascending = False)

,0
Invoice_ID,2600
Date,540
Total_Price,35
Unit_Price,16
Product,10
Customer_Name,7
Brand,7
Branch,4
Payment_Method,4
Quantity,3


###  Observation Summary:
The dataset contains a high number of unique invoices and dates, indicating many transactions over time. Most categorical columns like Product, Brand, Branch, and Payment_Method have limited unique values, reflecting a controlled and structured retail environment.


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Invoice_ID      2600 non-null   object
 1   Date            2600 non-null   object
 2   Customer_Name   2267 non-null   object
 3   Product         2600 non-null   object
 4   Brand           2600 non-null   object
 5   Quantity        2600 non-null   int64 
 6   Unit_Price      2600 non-null   int64 
 7   Branch          2600 non-null   object
 8   Payment_Method  1992 non-null   object
 9   Total_Price     2600 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 203.3+ KB


##  Data Summary and Observations

###  General Info:
- The dataset contains **2600 rows** and **10 columns**.
- This is sales data, including invoice info, products, prices, customers, and payment methods.

-  `Invoice_ID`: OK – All values are present and unique.
-  `Date`: Type is text, needs to be converted to datetime.
- `Customer_Name`: 333 missing – can fill with "Unknown".
-  `Product`: OK – No missing values.
-  `Brand`: OK – No missing values.
-  `Quantity`: OK – Numeric and complete.
-  `Unit_Price`: OK – Numeric and complete.
-  `Branch`: OK – No missing values.
-  `Payment_Method`: 608 missing – fill with "Not Recorded".
-  `Total_Price`: OK – Numeric and complete.

###  Next Steps:
- Clean missing values.
- Convert `Date` to datetime.
- Validate that `Total_Price = Quantity × Unit_Price`.

In [9]:
missing = df.isna().sum().sort_values(ascending = False)
print(f'Total Number of missing values in the Dataset {missing.sum()}\n')
missing = missing[missing>0]
missing

Total Number of missing values in the Dataset 941



,0
Payment_Method,608
Customer_Name,333


##  Missing Values

-  **Total Missing Values**: 941
- `Payment_Method`: 608 missing
  - Observation: Many transactions have no recorded payment method.

  
-  `Customer_Name`: 333 missing
  - Observation: Some invoices are missing customer names.

  - Action: you can check pattern

 All other columns have **zero missing values** — data is mostly clean.


## Analyze the pattern of missing data in `Customer_Name` and `Payment_Method`

In [10]:

def Check_Pattern(missing_df, null_col):
    """
    Analyze the pattern of missing data in a specific column.

    Parameters:
    ----------
    missing_df : pandas.DataFrame
        Rows where the specified column is missing.
    null_col : str
        The column with missing values.

    Returns:
    -------
    pandas.DataFrame
        Columns that tend to take only 1 or 2 unique values
        when `null_col` is missing — this may indicate a non-random pattern.
    """
    # Remove the target column (we don't need to analyze it here)
    missing_df = missing_df.drop(null_col, axis=1)

    # Create an empty DataFrame to store unique values for relevant columns
    unique_df = pd.DataFrame()

    for col in missing_df.columns:
        nunique = missing_df[col].nunique()

        if nunique in [1, 2]:  # If only 1 or 2 unique values exist
            unique_vals = missing_df[col].unique()
            unique_df[col + '_unique_vals'] = pd.Series(unique_vals)

    # Interpretation
    if unique_df.empty:
        print(f'\t- Missing values in `{null_col}` appear to be randomly distributed.')
        print(f'\t- Likely missing mechanism: MCAR (Missing Completely At Random)')
    else:
        print(f'\t- Missing values in `{null_col}` are associated with specific values in other columns:')
        print(f'\t  Columns with 1 or 2 unique values when `{null_col}` is missing: {list(unique_df.columns)}')
        print(f'\t- Likely missing mechanism: MAR or MNAR (Not Missing Completely At Random)')

    return unique_df


def Missing_Pattern(df, col):
    """
    Show stats and pattern analysis for a column with missing values.

    Parameters:
    ----------
    df : pandas.DataFrame
    col : str
        Column name to analyze
    """
    print(f"\nFeature: {col}")
    print('-'*40)
    print(f"\t- Number of missing values: {df[col].isna().sum()}")
    print(f"\t- Percentage of missing values: {(df[col].isna().mean())*100:.2f}%")
    print(f"\t- Data type: {df[col].dtype}")
    print(f"\t- Number of unique values: {df[col].nunique(dropna=True)}")
    print(f"\t- Most common value: {df[col].mode(dropna=True).iloc[0] if df[col].notna().any() else 'N/A'}")

    print(f"\nAnalyzing missing value pattern...")
    print('-'*40)
    missing = df[df[col].isna()]
    unique_df = Check_Pattern(missing, col)
    return missing, unique_df


In [11]:
missing1, pattern1 = Missing_Pattern(df, 'Customer_Name')
missing2, pattern2 = Missing_Pattern(df, 'Payment_Method')


Feature: Customer_Name
----------------------------------------
	- Number of missing values: 333
	- Percentage of missing values: 12.81%
	- Data type: object
	- Number of unique values: 7
	- Most common value: Mahmoud

Analyzing missing value pattern...
----------------------------------------
	- Missing values in `Customer_Name` appear to be randomly distributed.
	- Likely missing mechanism: MCAR (Missing Completely At Random)

Feature: Payment_Method
----------------------------------------
	- Number of missing values: 608
	- Percentage of missing values: 23.38%
	- Data type: object
	- Number of unique values: 4
	- Most common value: Mobile Wallet

Analyzing missing value pattern...
----------------------------------------
	- Missing values in `Payment_Method` appear to be randomly distributed.
	- Likely missing mechanism: MCAR (Missing Completely At Random)


### Missing Values Analysis Summary

- **Customer_Name**
  - ~12.8% missing values
  - Missing Completely At Random (MCAR)
  -  Action: Fill missing values with `'Unknown'` to keep the data and track anonymous customers.

- **Payment_Method**
  - ~23.4% missing values
  - Missing Completely At Random (MCAR)
  -  Action: Fill missing values with the most common value (`mode`), e.g., `'Mobile Wallet'`.

###  Final Decision:
- No rows will be dropped.
- Missing values will be imputed to retain useful sales data for analysis.

In [12]:
df['Customer_Name'].fillna('Unknown', inplace=True)

<ipython-input-12-1aedb04077b4>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Customer_Name'].fillna('Unknown', inplace=True)


In [13]:
df['Customer_Name'].isna().sum()

np.int64(0)

In [14]:
df['Payment_Method'].fillna(df['Payment_Method'].mode()[0], inplace=True)

<ipython-input-14-b6ef006aba43>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Payment_Method'].fillna(df['Payment_Method'].mode()[0], inplace=True)


In [15]:
df['Payment_Method'].isna().sum()

np.int64(0)

## Handle duplicates

In [16]:
df.duplicated().sum()

np.int64(0)

# Fix column data types if needed

In [17]:
df.dtypes

,0
Invoice_ID,object
Date,object
Customer_Name,object
Product,object
Brand,object
Quantity,int64
Unit_Price,int64
Branch,object
Payment_Method,object
Total_Price,int64


### Observation : Convert `Date` column


In [18]:
df['Date'].unique()

array(['01/26/2025', '2025-02-20', '2024-06-22', '2025-04-12',
       '2024-12-26', '2024-10-19', '2024-03-19', '2024-12-12',
       '2024-07-22', '2024-02-21', '2024-01-03', '2024-01-19',
       '2024-02-04', '2024-11-24', '2024-09-14', '2024-05-30',
       '2024-11-09', '2024-05-03', '2025-02-18', '2024-11-25',
       '2024-09-21', '2024-02-08', '2025-01-15', '2024-09-02',
       '2024-07-31', '2024-05-24', '2024-03-20', '2024-08-23',
       '2025-03-27', '2024-05-11', '2024-03-05', '2024-05-25',
       '2024-09-12', '2024-04-15', '2024-07-07', '2024-07-28',
       '2025-03-28', '2024-12-08', '2024-02-17', '2025-01-30',
       '2025-03-12', '2024-03-06', '2024-05-12', '2025-02-14',
       '2024-09-11', '2024-10-18', '2024-07-14', '2024-07-27',
       '2024-08-09', '2024-02-01', '2024-03-07', '2024-05-22',
       '2024-03-14', '2025-01-22', '2024-03-24', '2024-02-13',
       '2024-12-03', '2024-05-09', '2025-01-28', '2024-05-20',
       '2024-05-21', '2024-10-04', '2024-10-31', '2024-

In [19]:

from datetime import datetime

def clean_date(date_str):
    if pd.isnull(date_str):
        return None

    # Replace / with - for consistency
    date_str = str(date_str).replace('/', '-')

    # Possible date formats (more can be added if needed)
    formats = ['%Y-%m-%d', '%d-%m-%Y', '%m-%d-%Y', '%d-%b-%Y', '%Y/%m/%d', '%d/%m/%Y']

    for fmt in formats:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            return date_obj.strftime('%Y-%m-%d')  # Standardize format
        except:
            continue

    return None  # If all parsing fails

# Example: apply on a column called 'Date'
df['Date'] = df['Date'].apply(clean_date)


When I looked at the column for dates, I noticed that all the dates were not in the same format. This inconsistency is a significant issue.

We will aim to resolve this in the future.

In [20]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


In [21]:
df['Date'].isna().sum()

np.int64(0)

In [22]:
missing = df.isna().sum().sort_values(ascending = False)
print(f'Total Number of missing values in the Dataset {missing.sum()}\n')
missing = missing[missing>0]
missing

Total Number of missing values in the Dataset 0



,0


In [23]:
df.to_csv("cleaned_data.csv", index=False)

In [24]:
df

,Invoice_ID,Date,Customer_Name,Product,Brand,Quantity,Unit_Price,Branch,Payment_Method,Total_Price
0,PET-7000,2025-01-26,Salma,Pet Shampoo,Royal Canin,1,120,Alexandria,Visa,120
1,PET-7001,2025-02-20,Unknown,Pet Carrier,Hill's,3,350,6th October,Mobile Wallet,1050
2,PET-7002,2024-06-22,Salma,Canned Pet Food,Royal Canin,1,70,Cairo,Visa,70
3,PET-7003,2025-04-12,Salma,Pet Carrier,Purina,3,400,Port Said,Cash,1200
4,PET-7004,2024-12-26,Reem,Scratching Post,Purina,2,250,6th October,Mobile Wallet,500
...,...,...,...,...,...,...,...,...,...,...
2595,PET-9595,2024-04-19,Salma,Cat Litter,Whiskas,3,90,6th October,Cash,270
2596,PET-9596,2024-11-29,Nour,Canned Pet Food,Purina,3,70,Cairo,Mobile Wallet,210
2597,PET-9597,2024-04-14,Unknown,Canned Pet Food,Pedigree,3,70,Port Said,Mobile Wallet,210
2598,PET-9598,2024-12-26,Nour,Pet Carrier,Purina,1,300,6th October,Visa,300
